<a href="https://colab.research.google.com/github/zyfer416/Construction-analysis/blob/main/imageCorrectness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-magic opencv-python tensorflow keras numpy matplotlib

In [3]:
from google.colab import files

uploaded = files.upload()  # Let user upload an image
image_path = list(uploaded.keys())[0]  # Get the uploaded image filename
print(f"📂 Uploaded file: {image_path}")


Saving Building.jpg to Building.jpg
📂 Uploaded file: Building.jpg


In [4]:
import magic

def is_valid_image(image_path):
    mime = magic.Magic(mime=True)  # Create a Magic instance
    file_type = mime.from_file(image_path)  # Get file type

    # Check if the file is an image
    return file_type.startswith("image/")

# Validate the uploaded image
if is_valid_image(image_path):
    print("✅ Valid image file.")
else:
    print("❌ Invalid file! Please upload a valid image.")


✅ Valid image file.


In [5]:
import cv2

def is_blurry(image_path, threshold=100):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
    if img is None:
        print("❌ Unable to read the image.")
        return True

    laplacian_var = cv2.Laplacian(img, cv2.CV_64F).var()  # Compute Laplacian variance

    if laplacian_var < threshold:
        print(f"⚠️ Image is blurry (Laplacian variance: {laplacian_var:.2f})")
        return True
    else:
        print(f"✅ Image is clear (Laplacian variance: {laplacian_var:.2f})")
        return False

# Check blur level
is_blurry(image_path)


✅ Image is clear (Laplacian variance: 1606.15)


False

In [6]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

def is_construction_site(image_path):
    model = VGG16(weights="imagenet")  # Load pre-trained model

    # Preprocess the image
    image = load_img(image_path, target_size=(224, 224))
    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    image_array = preprocess_input(image_array)

    # Make prediction
    predictions = model.predict(image_array)
    decoded_predictions = decode_predictions(predictions, top=20)[0]

    # Print predictions
    print("\n🔹 Top Predictions:")
    for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
        print(f"{i+1}. {label}: {score*100:.2f}%")

    # Check if construction-related terms are in predictions
    construction_keywords = [
    "crane", "scaffold", "construction site", "bulldozer", "excavator", "altar", "building",
    "foundation work", "concrete pouring", "rebar reinforcement", "steel framework", "bricklaying",
    "masonry work", "construction workers", "heavy machinery", "road construction", "high-rise construction",
    "plastering work", "painting construction", "roof construction", "structural engineering",
    "construction site inspection", "tower crane", "earthmoving equipment", "site excavation",
    "bridge construction", "tunnel construction", "construction materials", "formwork setup",
    "site development", "pavement construction", "cement mixing", "construction debris"
]
    # The keyword needs to be defined and iterated over for each keyword in construction_keywords, label, score in the decoded_predictions list
    return any(keyword in label.lower() for keyword in construction_keywords for _, label, _ in decoded_predictions)

# Check relevance
if is_construction_site(image_path):
    print("✅ Relevant image: Construction site detected.")
else:
    print("⚠️ Image may not be relevant to construction.")


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 962ms/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

🔹 Top Predictions:
1. drilling_platform: 32.67%
2. palace: 13.36%
3. crane: 13.05%
4. dock: 5.33%
5. church: 5.24%
6. bell_cote: 4.18%
7. airship: 3.44%
8. castle: 1.57%
9. pier: 1.47%
10. cab: 1.35%
11. flagpole: 1.09%
12. worm_fence: 0.78%
13. fountain: 0.67%
14. obelisk: 0.65%
15. picket_fence: 0.64%
16. organ: 0.60%
17. seashore: 0.53%
18. maze: 0.48%
19. traffic_light: 0.48%
20. lakeside: 0.45%
✅ Relevant image: Construction site detected.
